# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [99]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [20]:
data = pd.read_csv('./../data/customer_product_sales.csv')

In [40]:
display(data.head())
display(data.shape)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


(68584, 7)

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [171]:
display(data.columns)

total_purchased = data.groupby(["CustomerID","ProductName"]).agg({'Quantity':np.sum})
display(total_purchased.head())

Index(['CustomerID', 'FirstName', 'LastName', 'SalesID', 'ProductID',
       'ProductName', 'Quantity'],
      dtype='object')

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [80]:
#table = pd.pivot_table(df, values='D', index=['A', 'B'],columns=['C'], aggfunc=np.sum)
table = pd.pivot_table(data,values= "Quantity",index = ["ProductName"],columns = ["CustomerID"]).fillna(0)

display(table.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [96]:
from scipy.spatial.distance import pdist, squareform

#pdist -> distancia por parejas
#euclidean -> distancia entre dos puntos
# LA traspuesta es para que se compute columnas por filas, es decir los usuarios por los productos. En la celda anterior
# esta de esta manera para q seea mas facil de ver

distances = pdist(table.T, 'euclidean')
distances_matrix = squareform(distances)
display(distances_matrix)

array([[  0.        ,  10.72380529,   9.64365076, ..., 197.94948851,
        218.8241303 , 203.79892051],
       [ 10.72380529,   0.        ,  10.67707825, ..., 197.34994299,
        218.62525014, 203.95342606],
       [  9.64365076,  10.67707825,   0.        , ..., 197.32460566,
        218.02981447, 203.80628057],
       ...,
       [197.94948851, 197.34994299, 197.32460566, ...,   0.        ,
        266.9269563 , 266.9269563 ],
       [218.8241303 , 218.62525014, 218.02981447, ..., 266.9269563 ,
          0.        , 269.25824036],
       [203.79892051, 203.95342606, 203.80628057, ..., 266.9269563 ,
        269.25824036,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [166]:
distances = pd.DataFrame(1/(1 + distances_matrix), index=table.columns, columns=table.columns)
display(distances.head())

#pongo 1:6, porque si pongo 0:5 el primero es él mismo
similarities = distances[33].sort_values(ascending=False)[1:6]
display(similarities.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903


CustomerID
3909    0.095358
264     0.093953
3531    0.093953
2503    0.093498
3305    0.093051
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [167]:

#hace una lista de los productos comprados por cada id
"""display(total_purchased.loc[33].head())

for name, score in dict(similarities).items():
    print(name,score)"""

list_series = [total_purchased.loc[name] for name, score in dict(similarities).items()]
top5 = pd.concat(list_series)

display(top5.head())
#display(top5.loc["Sage - Ground",])

,Quantity
ProductName,
Bay Leaf,2
"Beans - Kidney, Canned",1
Beer - Original Organic Lager,1
Beer - Rickards Red,1
Black Currants,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [168]:
top5_similar_costumer = top5.groupby("ProductName").sum().sort_values("Quantity",ascending=False)
display(top5_similar_costumer.head())


,Quantity
ProductName,
"Salsify, Organic",4
Quiche Assorted,3
Chocolate - Dark,3
Wine - Charddonnay Errazuriz,3
Bay Leaf,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [199]:
not_purchased = table[table[33]==0]

list_products = not_purchased.merge(top5_similar_costumer,left_on='ProductName', right_on='ProductName').sort_values("Quantity",ascending=False)
display(list_products.head())
similarities = list_products[[33,"Quantity"]]
similarities = similarities[0:5]
display(similarities)

,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
ProductName,,,,,,,,,,,,,,,,,,,,,
"Oranges - Navel, 72",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
Chocolate - Dark,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,0.0,0.0,0.0,3
Pecan Raisin - Tarts,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,3
Bay Leaf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,0.0,0.0,0.0,3
Wine - Charddonnay Errazuriz,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,25.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,3


,33,Quantity
ProductName,,
"Oranges - Navel, 72",0.0,3
Chocolate - Dark,0.0,3
Pecan Raisin - Tarts,0.0,3
Bay Leaf,0.0,3
Wine - Charddonnay Errazuriz,0.0,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [268]:
#obtengo los productos vacios
lst = set(data.ProductName.to_list())
dic_products = {}
for e in lst:
    dic_products[e] = ""

#obtengo el numeero de usuarios
users = data["CustomerID"].unique()

dic = {}

for user in lst_users:
    #distances = pd.DataFrame(1/(1 + distances_matrix), index=table.columns, columns=table.columns)
    #otengo los top5 de cada usuario
    similarities = distances[user].sort_values(ascending=False)[1:6]
    #obtengo el top5 comprado por cada user
    list_series = [total_purchased.loc[name] for name, score in dict(similarities).items()]
    top5 = pd.concat(list_series)
    
    #obtengo las recomendaciones
    not_purchased = table[table[user]==0]
    list_products = not_purchased.merge(top5_similar_costumer,left_on='ProductName', right_on='ProductName').sort_values("Quantity",ascending=False)
    list_products = list_products[0:5].index
                                 
    dic[user] = list_products
    

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [273]:
recommendations_all = pd.DataFrame.from_dict(dic)
display(recommendations_all.head())

,61288,77352,40094,23548,78981,83106,11253,35107,15088,26031,...,25270,17390,97029,38154,74403,49005,41286,85878,68506,3472
0,"Cheese - Boursin, Garlic / Herbs",Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,"Cheese - Boursin, Garlic / Herbs",Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,...,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap,Muffin - Carrot Individual Wrap
1,Pork - Kidney,Pork - Kidney,"Cheese - Boursin, Garlic / Herbs",Ice Cream Bar - Hageen Daz To,"Cheese - Boursin, Garlic / Herbs","Cheese - Boursin, Garlic / Herbs","Cheese - Boursin, Garlic / Herbs","Cheese - Boursin, Garlic / Herbs",Pork - Kidney,Pork - Kidney,...,Pork - Kidney,"Cheese - Boursin, Garlic / Herbs",Pork - Kidney,"Cheese - Boursin, Garlic / Herbs","Cheese - Boursin, Garlic / Herbs",Pork - Kidney,"Cheese - Boursin, Garlic / Herbs","Cheese - Boursin, Garlic / Herbs","Cheese - Boursin, Garlic / Herbs","Cheese - Boursin, Garlic / Herbs"
2,Beef Ground Medium,"Cheese - Boursin, Garlic / Herbs",Pork - Kidney,Sausage - Liver,Pork - Kidney,Pork - Kidney,Pork - Kidney,Pork - Kidney,Sausage - Liver,"Cheese - Boursin, Garlic / Herbs",...,Beef Wellington,Pork - Kidney,Beef Wellington,Ice Cream Bar - Hageen Daz To,Raspberries - Fresh,"Cheese - Boursin, Garlic / Herbs",Pork - Kidney,Pork - Kidney,Pork - Kidney,Pork - Kidney
3,"Pasta - Detalini, White, Fresh",Beef Ground Medium,Kellogs Special K Cereal,Beef Wellington,"Beans - Kidney, Canned","Pasta - Detalini, White, Fresh",Beef Ground Medium,Raspberries - Fresh,"Beans - Kidney, Canned",Raspberries - Fresh,...,"Pasta - Detalini, White, Fresh","Pasta - Detalini, White, Fresh",Raspberries - Fresh,"Pasta - Detalini, White, Fresh",Ice Cream Bar - Hageen Daz To,Raspberries - Fresh,"Beans - Kidney, Canned","Pasta - Detalini, White, Fresh",Raspberries - Fresh,"Beans - Kidney, Canned"
4,"Beans - Kidney, Canned",Beef Wellington,"Pasta - Detalini, White, Fresh",Kellogs Special K Cereal,Raspberries - Fresh,"Beans - Kidney, Canned",Ice Cream Bar - Hageen Daz To,Beef Ground Medium,Kellogs Special K Cereal,"Pasta - Detalini, White, Fresh",...,Beef Ground Medium,Raspberries - Fresh,"Beans - Kidney, Canned",Sausage - Liver,Kellogs Special K Cereal,Knife Plastic - White,Kellogs Special K Cereal,Raspberries - Fresh,Ice Cream Bar - Hageen Daz To,Knife Plastic - White


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [276]:



def recommendations_users(metric):
    dic = {}
    users = data["CustomerID"].unique()
    for user in lst_users:
        distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, metric))), 
                         index=table.columns, columns=table.columns)
        #otengo los top5 de cada usuario
        similarities = distances[user].sort_values(ascending=False)[1:6]
        #obtengo el top5 comprado por cada user
        list_series = [total_purchased.loc[name] for name, score in dict(similarities).items()]
        top5 = pd.concat(list_series)

        #obtengo las recomendaciones
        not_purchased = table[table[user]==0]
        list_products = not_purchased.merge(top5_similar_costumer,left_on='ProductName', right_on='ProductName').sort_values(metric,ascending=False)
        list_products = list_products[0:5].index

        dic[user] = list_products
        


display(pd.DataFrame.from_dict(recommendations_users("cityblock")))
    

KeyError: 'cityblock'